# Drone Simulation

Example run. This run just uses a simple PD control.

In [ ]:
from itertools import cycle
import time
import pybullet as p
from drone_world import World, QuadPDController


## Note for MacOS users

The simulation software package that I use does not play nicely with Macos
and causes a lot of "hanging processes".

After running a GUI visuallization, you will need to restart the Jupyter 
Notebook Kernel to spawn a new "working" GUI session.

Skip this next cell if you just want to see the training.

In [ ]:

world = World(controller_type=QuadPDController, connection_type=p.GUI)

waypoints = cycle([
    (0, 0, 1),
    (1, 0, 2),
    (1, 1, 1),
    (0, 1, 2),
    (1, 0, 1),
    (1, 1, 2),
])

try:
    for k in range(30 * 240):
        # change target when waypoint is reached
        if world.reached():
            waypoint = next(waypoints)
            print("going to waypoint", waypoint)
            world.set_target(waypoint)

        world.step()
finally: 
    world.end()

# Can can use this to log our controls

This can create example data for our model.

In [ ]:

import numpy as np


# Create new world using a "direct" connection to skip GUI.
# Record controls to use as training data
world = World(controller_type=QuadPDController, connection_type=p.DIRECT, record_controls=True)

# 500 points, 3 coords: x,y ~ [-5,5), z ~ [1,2)
xs = np.load("xs.npy")
ys = np.load("ys.npy")
zs = np.load("zs.npy")

waypoints = cycle(zip(xs, ys, zs))

for k in range(30 * 240):
    # change target when waypoint is reached
    if world.reached():
        waypoint = next(waypoints)
        print("going to waypoint", waypoint)
        world.set_target(waypoint)

    world.step()
world.end()

print(world.controls[0])
# We get a big list of controls!

# Define a NN to learn from our controls

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split

# ---------- 1) Feature extraction ----------
def sample_to_xy(sample):
    s, t = sample["state"], sample["target"]
    # state: pos(3), vel(3), rpy(3), R(3x3), w_body(3) = 21
    pos   = np.asarray(s["pos"], dtype=np.float64).ravel()
    vel   = np.asarray(s["vel"], dtype=np.float64).ravel()
    rpy   = np.asarray(s["rpy"], dtype=np.float64).ravel()
    R     = np.asarray(s["R"], dtype=np.float64).ravel()        # 9
    w     = np.asarray(s["w_body"], dtype=np.float64).ravel()
    # target: pos(3) + yaw(1) = 4
    tpos  = np.asarray(t["pos"], dtype=np.float64).ravel()
    yaw   = np.array([t["yaw"]], dtype=np.float64)

    x = np.concatenate([pos, vel, rpy, R, w, tpos, yaw])  # shape (25,)
    y = np.asarray(sample["thrusts"], dtype=np.float64).ravel()  # shape (4,)
    return x, y

# ---------- 2) Dataset ----------
class ThrustDataset(Dataset):
    def __init__(self, data_list):
        X, Y = zip(*(sample_to_xy(s) for s in data_list))
        self.X = np.stack(X).astype(np.float32)  # (N, 25)
        self.Y = np.stack(Y).astype(np.float32)  # (N, 4)

        # simple standardization (save stats for inference)
        self.x_mean = self.X.mean(axis=0, keepdims=True)
        self.x_std  = self.X.std(axis=0, keepdims=True) + 1e-8
        self.y_mean = self.Y.mean(axis=0, keepdims=True)
        self.y_std  = self.Y.std(axis=0, keepdims=True) + 1e-8

        self.Xn = (self.X - self.x_mean) / self.x_std
        self.Yn = (self.Y - self.y_mean) / self.y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.Xn[idx])   # (25,)
        y = torch.from_numpy(self.Yn[idx])   # (4,)
        return x, y

# ---------- 3) Model (tanh MLP) ----------
class TanhMLP(nn.Module):
    def __init__(self, in_dim=25, out_dim=4, hidden=(128, 128)):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers += [nn.Linear(last, h), nn.Tanh()]
            last = h
        layers += [nn.Linear(last, out_dim)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ---------- 4) Training utility ----------
def train_model(data_list, epochs=100, batch_size=128, lr=1e-3, val_split=0.2, hidden=(128,128), device="cpu"):
    ds = ThrustDataset(data_list)
    n_val = int(len(ds) * val_split)
    n_tr  = len(ds) - n_val
    tr_ds, val_ds = random_split(ds, [n_tr, n_val], generator=torch.Generator().manual_seed(0))

    tr_loader  = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    model = TanhMLP(in_dim=ds.X.shape[1], out_dim=ds.Y.shape[1], hidden=hidden).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    def run_epoch(loader, train=True):
        model.train(train)
        total = 0.0
        with torch.set_grad_enabled(train):
            for xb, yb in loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss = loss_fn(pred, yb)
                if train:
                    opt.zero_grad()
                    loss.backward()
                    opt.step()
                total += loss.item() * xb.size(0)
        return total / len(loader.dataset)

    for ep in range(1, epochs+1):
        tr_loss = run_epoch(tr_loader, train=True)
        val_loss = run_epoch(val_loader, train=False) if n_val > 0 else float('nan')
        if ep % 5 == 0 or ep == 1 or ep == epochs:
            print(f"Epoch {ep:3d} | train MSE(norm): {tr_loss:.5f}" + (f" | val: {val_loss:.5f}" if n_val > 0 else ""))

    # package normalization for inference
    norm = {
        "x_mean": torch.from_numpy(ds.x_mean.astype(np.float32)),
        "x_std":  torch.from_numpy(ds.x_std.astype(np.float32)),
        "y_mean": torch.from_numpy(ds.y_mean.astype(np.float32)),
        "y_std":  torch.from_numpy(ds.y_std.astype(np.float32)),
    }
    return model, norm

# ---------- 5) Inference helper (denormalize output) ----------
@torch.no_grad()
def predict_thrusts(model, norm, samples):
    xs = np.stack([sample_to_xy(s)[0] for s in samples]).astype(np.float32)  # (B,25)
    Xn = (xs - norm["x_mean"].numpy()) / norm["x_std"].numpy()
    Xn = torch.from_numpy(Xn)
    yn = model(Xn)
    y  = yn * norm["y_std"] + norm["y_mean"]
    return y.numpy()  # (B,4)

# Suppose `data` is your list of dicts like the one you showed
model, norm = train_model(world.controls, epochs=150, batch_size=256, lr=1e-3, hidden=(128,128))

# Predict on a few samples
pred = predict_thrusts(model, norm, world.controls[:5])  # shape (5,4)
print(pred)

# Now we have a simple model that learned to copy our PD Controller!

Lets visuallize its outputs!

## But first, we need to define a controller class that uses our trained model

In [ ]:
from drone_world import Controller, Drone

def build_features(state, target) -> np.ndarray:
    s, t = state, target
    pos   = np.asarray(s["pos"], dtype=np.float64).ravel()
    vel   = np.asarray(s["vel"], dtype=np.float64).ravel()
    rpy   = np.asarray(s["rpy"], dtype=np.float64).ravel()
    R     = np.asarray(s["R"], dtype=np.float64).ravel()          # (9,)
    w     = np.asarray(s["w_body"], dtype=np.float64).ravel()

    tpos  = np.asarray(t["pos"], dtype=np.float64).ravel()
    yaw   = np.array([t["yaw"]], dtype=np.float64)

    x = np.concatenate([pos, vel, rpy, R, w, tpos, yaw]).astype(np.float32)  # (25,)
    return x


class LearnedController(Controller):
    def __init__(self, drone: Drone) -> None:
        self.x_mean = norm["x_mean"].float().cpu().numpy()  # (1,25)
        self.x_std  = norm["x_std"].float().cpu().numpy()   # (1,25)
        self.y_mean = norm["y_mean"].float().cpu().numpy()  # (1,4)
        self.y_std  = norm["y_std"].float().cpu().numpy()   # (1,4)


    def update(self, state, target) -> tuple:
        x = build_features(state, target)                       # (25,)
        xn = (x - self.x_mean.squeeze(0)) / self.x_std.squeeze(0)

        # 2) model forward (normalized space)
        xb = torch.from_numpy(xn[None, :])

        with torch.no_grad():
            yn = model(xb).cpu().numpy()[0]
            y = yn * self.y_std.squeeze(0) + self.y_mean.squeeze(0)  # (4,)
            return tuple(float(v) for v in y), (0, 0, 0, 0) 

In [ ]:
world = World(controller_type=LearnedController, connection_type=p.GUI)

waypoints = cycle([
    (0, 0, 1),
    (1, 0, 2),
    (1, 1, 1),
    (0, 1, 2),
    (1, 0, 1),
    (1, 1, 2),
])

try:
    for k in range(30 * 240):
        # change target when waypoint is reached
        if world.reached():
            waypoint = next(waypoints)
            print("going to waypoint", waypoint)
            world.set_target(waypoint)

        world.step()
finally: 
    world.end()

We can see this model doesn't work very well, but this is just a demonstration
of our *workflow*, not any of the models we actually intend to train.